<a href="https://colab.research.google.com/github/stephenbeckr/convex-optimization-class/blob/main/Demos/CVX_demo/tutorialSolutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python solutions for `cvxpy` exercise

You can see what's in the output of `prob` by doing `vars(prob.solver_stats)`

The default solver depends on what kind of problem you're solving, but it's often `ECOS`.  See [advanced cvxpy settings](https://www.cvxpy.org/tutorial/advanced/index.html#setting-solver-options) for how to change accuracy options.  For `ECOS` the relevant options are `abstol` and `reltol` (defaults are 1e-8).

This notebook is for [https://github.com/stephenbeckr/convex-optimization-class/tree/master/Demos/CVX_demo](https://github.com/stephenbeckr/convex-optimization-class/tree/master/Demos/CVX_demo)

In [3]:
import numpy as np # np.array (and used internally in cvxpy)
import cvxpy as cvx
import sys, time

print("Using CVX version", cvx.__version__)
print(" and python version", sys.version)

A = np.array([[1, 6,11, 5,10, 4, 9, 3, 8, 2],
              [2, 7, 1, 6,11, 5,10, 4, 9, 3],
              [3, 8, 2, 7, 1, 6,11, 5,10, 4],
              [4, 9, 3, 8, 2, 7, 1, 6,11, 5],
              [5,10, 4, 9, 3, 8, 2, 7, 1, 6]])

# or a slicker way to create A
A = np.tile( np.arange(1,12), 5 )[:-5].reshape((5,-1),order='F')
# Use np.tile( a, 5 ) or np.concatenate( (a,a,a,a,a) )
# then remove last 5 elements (want 50 elements, not 55), then reshape:
# "Both the numpy.reshape() and numpy.resize() methods are used to change the
#  size of a NumPy array. The difference between them is that the reshape()
#  does not changes the original array but only returns the changed array,
#  whereas the resize() method returns nothing and directly changes the original array."
#
# And either use Fortran order 'F' or theh default 'C' order but make it size (10,5) and then transpose that.


print(A.shape)

y = np.array([1,2,3,4,5])     # size (5,)  [preferred]
# y = np.array([[1,2,3,4,5]]).T # size (5,1)  [may lead to bugs if you don't use y.flatten() later]
print(y.shape)

tol = 1e-12

def print_status(prob, x):
   print("Problem status: ", prob.status);
   print("Used the solver: ",
         prob.solver_stats.solver_name, "with",
         prob.solver_stats.num_iters, "iterations.")
   print("Optimal value:  ", prob.value)
   print("Optimal var:\n", x.value)

Using CVX version 1.6.0
 and python version 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
(5, 10)
(5,)


## Problem 1

In [5]:
x = cvx.Variable(10) # column vector with 10 elements

obj = cvx.Minimize(cvx.norm(x)) # cvx.norm defaults to the 2-norm
#constraints = [cvx.norm(A*x-y) <= 0.1] # specify a list of constraints
constraints = [cvx.norm(A@x-y) <= 0.1] # New syntax

prob = cvx.Problem(obj, constraints)
prob.solve()
# prob.solve(abstol=tol,reltol=tol) # this used to work, depends on what the default solver is

print_status(prob, x)

Problem status:  optimal
Used the solver:  CLARABEL with 7 iterations.
Optimal value:   0.2942164969709008
Optimal var:
 [ 0.08864839  0.1397823  -0.03834094  0.12955552 -0.04299356  0.11932874
 -0.04291282  0.10910196 -0.03608625  0.09887517]


## Problem 2

In [6]:
x = cvx.Variable(10)

obj = cvx.Minimize(cvx.norm(x)**2) # cvxpy objects implement the standard python ops
constraints = [cvx.norm(A@x-y) <= 0.1]

prob = cvx.Problem(obj, constraints)
prob.solve()#abstol=tol,reltol=tol)

print_status(prob, x)

Problem status:  optimal
Used the solver:  CLARABEL with 8 iterations.
Optimal value:   0.08656333586620392
Optimal var:
 [ 0.08864834  0.13978235 -0.03833756  0.12955555 -0.04299696  0.11932875
 -0.04291652  0.10910194 -0.03608115  0.09887514]


## Problem 3

In [7]:
x = cvx.Variable(10)

obj = cvx.Minimize(cvx.norm(x, p=1))
constraints = [cvx.norm(A@x-y) <= 0.1]

prob = cvx.Problem(obj, constraints)
prob.solve()#abstol=tol,reltol=tol)

print_status(prob, x)

Problem status:  optimal
Used the solver:  CLARABEL with 7 iterations.
Optimal value:   0.7876692205253679
Optimal var:
 [ 8.31157556e-10  5.51546139e-01 -5.62018042e-02  9.98730289e-09
 -6.22342246e-02  2.34742979e-09 -6.22342071e-02  1.25338157e-09
 -5.54528305e-02  9.61489775e-10]


## Problem 4

In [9]:
def get_problem1_dual_value():
  obj = cvx.Minimize(cvx.norm(x))
  constraints = [cvx.norm(A@x-y) <= 0.1]

  prob = cvx.Problem(obj, constraints)
  prob.solve()#abstol=tol,reltol=tol)

  return constraints[0].dual_value

x = cvx.Variable(10)

# resolve problem 1 and return dual value for the constraint
l = get_problem1_dual_value()
print("dual variable: ", l)

obj = cvx.Minimize(cvx.norm(x) + l*cvx.norm(A@x-y))

prob = cvx.Problem(obj)
prob.solve()#abstol=tol,reltol=tol)

# note that the solution is the same, but the optimal value is different,
# since for problem 1 we form the Lagrangian \|x\|_2 + \lambda(\|Ax-y\|_2-0.1)
print_status(prob, x)

# the optimal value for problem 1 should be
print("problem 1 optimal value: ", prob.value - 0.1*l)

dual variable:  0.07086818142507507
Problem status:  optimal
Used the solver:  CLARABEL with 9 iterations.
Optimal value:   0.3013032949384033
Optimal var:
 [ 0.08865406  0.13978924 -0.03834808  0.1295622  -0.04299771  0.11933517
 -0.04291869  0.10910813 -0.03609731  0.0988811 ]
problem 1 optimal value:  0.2942164767958958


## Problem 5

In [10]:
# x = cvx.Variable(5)  # we need to make this an explicit column vector
x = cvx.Variable((5,1))
ones = np.ones((10,1))

obj = cvx.Minimize(sum(cvx.norm(A-x@ones.T, axis=0))) # cvx.norm behaves like np.linalg.norm

prob = cvx.Problem(obj)
prob.solve()#abstol=tol,reltol=tol)

print_status(prob, x)

Problem status:  optimal
Used the solver:  CLARABEL with 10 iterations.
Optimal value:   63.955138722823264
Optimal var:
 [[4.53780497]
 [5.05919913]
 [5.64349739]
 [6.2352254 ]
 [6.78779717]]


## Problem 6

In [11]:
x = cvx.Variable((5,1)) # again, be careful here
ones = np.ones((10,1))

obj = cvx.Minimize(cvx.norm(A-x@ones.T))

prob = cvx.Problem(obj)
# ECOS solver won't solve this kind of thing. SCS is the new default
prob.solve(verbose=False,eps=tol) # ~1e-7 duality gap, but CVXOPT gets a singular KKT system
print_status(prob, x)
prob.solve(verbose=False, solver='CVXOPT', kktsolver='robust')
print_status(prob, x)

Problem status:  optimal
Used the solver:  SCS with 250 iterations.
Optimal value:   14.392200222565954
Optimal var:
 [[5.90000493]
 [5.80000174]
 [5.7000011 ]
 [5.60000385]
 [5.5000082 ]]
Problem status:  optimal
Used the solver:  CVXOPT with None iterations.
Optimal value:   14.392200222565954
Optimal var:
 [[5.9]
 [5.8]
 [5.7]
 [5.6]
 [5.5]]


## Problem 7

In [12]:
# X = cvx.Variable(5,10) # old syntax
X = cvx.Variable((5,10))

obj = cvx.Minimize(cvx.norm(X-A, 'fro'))
constraints = [ np.ones((5,)).T@X@np.ones((10,)) == 1. ]

prob = cvx.Problem(obj, constraints)
prob.solve()

print_status(prob, X)

Problem status:  optimal
Used the solver:  CLARABEL with 5 iterations.
Optimal value:   40.163665171395905
Optimal var:
 [[-4.68  0.32  5.32 -0.68  4.32 -1.68  3.32 -2.68  2.32 -3.68]
 [-3.68  1.32 -4.68  0.32  5.32 -0.68  4.32 -1.68  3.32 -2.68]
 [-2.68  2.32 -3.68  1.32 -4.68  0.32  5.32 -0.68  4.32 -1.68]
 [-1.68  3.32 -2.68  2.32 -3.68  1.32 -4.68  0.32  5.32 -0.68]
 [-0.68  4.32 -1.68  3.32 -2.68  2.32 -3.68  1.32 -4.68  0.32]]


## Problem 8

In [13]:
B = A[:,0:5]
X = cvx.Variable((5,5)) # could use Semidef or Symmetric here instead

obj = cvx.Minimize(cvx.norm(X-B, 'fro'))
constraints = [ X == X.T, X >> 0 ] # X is PSD

prob = cvx.Problem(obj, constraints)
prob.solve()

print_status(prob, X)

Problem status:  optimal
Used the solver:  SCS with 50 iterations.
Optimal value:   14.435946533119205
Optimal var:
 [[4.64375748 5.21368245 4.26848539 4.75864117 4.86465025]
 [5.21368243 9.09234036 4.13923569 7.10383591 7.80302294]
 [4.26848537 4.13923571 4.22624558 4.64923409 3.88386332]
 [4.75864117 7.10383592 4.64923408 8.15616309 5.83270695]
 [4.86465023 7.80302294 3.88386333 5.83270697 6.8664761 ]]


## Problem 9

In [18]:
print('Rerun Problem 1 without parameterizing ...')
x = cvx.Variable(10)
obj = cvx.Minimize(cvx.norm(x))
constraints = [cvx.norm(A@x - y) <= 0.1]
prob = cvx.Problem(obj, constraints)
t = time.time()
prob.solve()
elapsed = time.time() - t
print(f"  Elapsed time: {elapsed} seconds.")
print('Now parameterize y ...')
b = cvx.Parameter(5)
obj = cvx.Minimize(cvx.norm(x))
constraints = [cvx.norm(A@x - b) <= 0.1]
prob = cvx.Problem(obj, constraints)

elapsed = 0.
for i in range(10):
  b.value = np.random.rand(5)
  t = time.time()
  prob.solve()
  elapsed += time.time() - t
print(f"  Avg elapsed time: {elapsed} seconds.")

Rerun Problem 1 without parameterizing ...
  Elapsed time: 0.009446144104003906 seconds.
Now parameterize y ...
  Avg elapsed time: 0.023536205291748047 seconds.
